In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = "./test_integration.csv"
df = pd.read_csv(data_path)

#### Get the accuracy of grader agent

In [5]:
label_map = {
    0: '<4',
    1: 4.0,
    2: 4.5,
    3: 5.0,
    4: 5.5,
    5: 6.0,
    6: 6.5,
    7: 7.0,
    8: 7.5,
    9: 8.0,
    10: 8.5,
    11: 9.0
}

def get_acc_with_tol(pred_list, truth_list):
    total = len(truth_list)
    correct = 0
    for (pred, truth) in zip(pred_list, truth_list):
        if '<4' in pred:
            pred = "<4"
        elif float(pred) <= 4:
            pred = '<4'
        try:
            if pred == '<4' or truth == '<4':
                if pred == truth:
                    correct += 1 
            elif float(pred) == float(truth) or abs(float(pred) - float(truth)) == 0.5:
            #elif float(pred) == float(truth) or abs(float(pred) - float(truth)) <= 1.0:
                correct += 1
        except:
            total -= 1
            print(f"Conversion Error, Skipping pred: {pred}, truth: {truth}")
            continue
    return correct / total


def get_acc_no_tol(pred_list, truth_list):
    total = len(truth_list)
    correct = 0
    for (pred, truth) in zip(pred_list, truth_list):
        if '<4' in pred:
            pred = "<4"
        elif float(pred) < 4:
            pred = '<4'
        try:
            if pred == '<4' or truth == '<4':
                if pred == truth:
                    correct += 1
            elif float(pred) == float(truth):
                correct += 1
        except:
            total -= 1
            print(f"Conversion Error, Skipping pred: {pred}, truth: {truth}")
            continue
    return correct / total

In [9]:
predict_list = df["predicted"]
truth_list = [label_map[i] for i in df["label"]]

print(" ======== Grader LLM Performance Summary ===========\n")
print(f"accuracy with no tolerace: {get_acc_no_tol(predict_list, truth_list)}")
print(f"accuracy with 0.5 tolerace: {get_acc_with_tol(predict_list, truth_list)}\n")


 ======== Grader LLM Performance Summary ===========

accuracy with no tolerace: 0.22448979591836735
accuracy with 0.5 tolerace: 0.5204081632653061



### Get the performance of the sample essay generator

In [15]:
sample_essay_list = df["sample_essay"]
failed_essay_num = 0
for e in sample_essay_list:
    if "Sorry, we cannot generate" in e:
        failed_essay_num += 1
    
print(f"{failed_essay_num} failed to generate a sample essay")
print(f"{(len(sample_essay_list) - failed_essay_num) / len(sample_essay_list) * 100} %")


4 failed to generate a sample essay
95.91836734693877 %


### Get the performance of the feedback generator

In [18]:
feedback_list = df["feedback"]
failed_feedback_num = 0
for f in feedback_list:
    if "But I failed to provide" in f or "Failed to generate 'good' feedback" in f:
        failed_feedback_num += 1

print(f"{failed_feedback_num} failed to generate feedback")
print(f"{(len(feedback_list) - failed_feedback_num) / len(feedback_list) * 100} %")


1 failed to generate feedback
98.9795918367347 %
